<a id="top"></a>
# Composites
<hr>

## Background

Composites are 2-dimensional representations of 3-dimensional data.
There are many cases in which this is desired. Sometimes composites are used in visualization - such as showing an RGB image of an area. Other times they are used for convenience, such as reducing the run time of an analysis by reducing the amount of data to be processed in a task by working with composites instead of full datasets. Other times they are required by an algorithm.

There are several kinds of composites that can be made. This notebook provides an overview of several of them and shows how to create them in the context of Open Data Cube.
<hr>

## Index

* [Import Dependencies and Connect to the Data Cube](#Composites_import)
* [Load Data from the Data Cube](#Composites_retrieve_data)
* [Most Common Composites](#Composites_most_common)
    * Mean composites
    * Median composites
    * Geometric median (geomedian) composites
    * Geometric medoid (geomedoid) composites
* [Other Composites](#Composites_other_composites)
    * Most-recent composites
    * Least-recent composites

## <span id="Composites_import">Import Dependencies and Connect to the Data Cube [&#9652;](#top)</span>

In [1]:
import sys
import os
sys.path.append(os.environ.get('NOTEBOOK_ROOT'))

from utils.data_cube_utilities.clean_mask import landsat_qa_clean_mask, landsat_clean_mask_invalid
from utils.data_cube_utilities.dc_mosaic import create_hdmedians_multiple_band_mosaic
from utils.data_cube_utilities.dc_mosaic import create_mosaic

import datacube
dc = datacube.Datacube()

## <span id="Composites_retrieve_data">Load Data from the Data Cube [&#9652;](#top)</span>

In [2]:
product = "ls8_lasrc_kenya"
platform = "LANDSAT_8"

In [3]:
landsat_ds = dc.load(platform=platform, product=product,
                     time=("2017-01-01", "2017-12-31"), 
                     lat=(-1.395447, -1.172343), 
                     lon=(36.621306, 37.033980),
                     group_by='solar_day',
                     dask_chunks={'latitude':500, 'longitude':500,
                                  'time':5})

In [4]:
clean_mask = (landsat_qa_clean_mask(landsat_ds, platform) & 
              (landsat_ds != -9999).to_array().all('variable') & 
              landsat_clean_mask_invalid(landsat_ds))

In [5]:
landsat_ds = landsat_ds.where(clean_mask)

## <span id="Composites_most_common">Most Common Composites [&#9652;](#top)</span>

### Mean composites

> A mean composite is obtained by finding the mean (average) value of each band for each pixel. To create mean composites, we use the built-in `mean()` method of xarray objects.

In [6]:
mean_composite = landsat_ds.mean('time', skipna=True)

### Median composites

> A median composite is obtained by finding the median value of each band for each pixel. Median composites are quick to obtain and are usually fairly representative of their data, so they are acceptable for visualization as images. To create median composites, we use the built-in `median()` method of xarray objects.

![](diagrams/urbanization/median_comp.png)  

In [7]:
median_composite = landsat_ds.median('time', skipna=True)

### Geometric median (geomedian) composites

> Geometric median (or "geomedian") composites are the best composites to use for most applications for which a representative, **synthetic** (calculated, not selected from the data) time slice is desired. They are essentiall median composites, but instead of finding the median on a per-band basis, they find the median for all bands together. If a composite will be used for analysis - not just visualization - it should be a geomedian composite. The only downside of this composite type is that it takes much longer to obtain than other composite types. For more information, see the [Geomedians_and_Geomedoids notebook](Geomedians_and_Geomedoids.ipynb).

In [8]:
geomedian_composite = create_hdmedians_multiple_band_mosaic(landsat_ds)

### Geometric medoid (geomedoid) composites

> Geometric medoid (or "geomedoid") composites are the best composites to use for most applications for which a representative, **non-syntheic** (selected from the data, not calculated) time slice is desired. For more information, see the [Geomedians_and_Geomedoids notebook](Geomedians_and_Geomedoids.ipynb).

In [9]:
geomedoid_composite = create_hdmedians_multiple_band_mosaic(landsat_ds, operation='medoid')

## <span id="Composites_other_composites">Other Composites [&#9652;](#top)</span>

### Most-recent composites

> Most-recent composites use the most recent cloud-free pixels in an image. To create, a most-recent composite, we use the **create_mosaic** utility function.

![](diagrams/urbanization/flat_mosaic.png)

In [10]:
most_recent_composite = create_mosaic(landsat_ds)

### Least-recent composites

> Least-recent composites are simply the opposite of most-recent composites. To create, a least-recent composite, we use the **create_mosaic** utility function, specifying `reverse_time=True`.

In [11]:
most_recent_composite = create_mosaic(landsat_ds, reverse_time=True)